# Thoth 0.5.0 - Example 1 Guided Notebook - Lockdown

One of the basic tasks of a developer is to keep the application’s dependencies up to date. In the Python ecosystem tools like `pip` and `Pipenv` help with that task. As a best practice, lock files have emerged. These lock files lock the version of all dependencies (including transitive dependencies) to specific versions. The primary goal is to provide all information required for deterministic builds of the application.
This use case is supported by Thoth’s command line tool [Thamos](https://pypi.org/project/thamos): Thamos will read an application’s Pipfile, submit it to Thoth’s API and based on the received result it will create a `Pipfile.lock`. The same goal can be achieved by using `pipenv lock`.

See internal document for more info and clarification in [Google Docs](https://docs.google.com/document/d/1QflQpGXtOuHFFC2hkEFBlu0JmCQWEnXdgryvwxCNXpQ/edit#) - section "Example 1".

This notebook is a supportive material for Thoth 0.5.0 release and use case stated above - it walks through the first example scenario from within a Jupyter Notebook. The same can be accomplished using `Thamos` CLI tool, as stated above, following instructions present at the bottom of this notebook.

In order to go through this scenario, we need first connect to a graph database instance. This notebook is playeble from within your computer, it inserts all the data into a provided JanusGraph instasnce, so select a graph database instance you would like to use. If you want to run this script purely on your local machine, setup your local graph database as [described in the README file of janusgraph-thoth-config repo](https://github.com/thoth-station/janusgraph-thoth-config#running-janusgraph-instance-locally). Ideally, just clone the repo and issue the following command to setup your local JanusGraph database instance:

```
sudo ./local.sh all
```

In [1]:
# Configure JanusGraph instance to talk to:
JANUSGRAPH_SERVICE_HOST = 'localhost'

# For directly talking to test environment, uncomment the following line:
#JANUSGRAPH_SERVICE_HOST = 'janusgraph.test.thoth-station.ninja'

Now let's connect to desired JanusGraph database and check if we are properly connected:

In [2]:
from thoth.storages import GraphDatabase

# Instantiate and connect the JanusGraph database.
graph = GraphDatabase.create(JANUSGRAPH_SERVICE_HOST)
graph.connect()

graph.is_connected()

True

To let Thoth give you guidenance, there is a need to have some data precomputed. One of such data are solver results which resolve dependencies and observe whether packages are installable into a specific environment, what are dependencies of packages and how these dependencies are structured respecting transitive dependency graphs.

There is [prepared a solver result in a form of a JSON](https://github.com/thoth-station/misc/tree/master/examples/lockdown). The solver is run in Thoth deployment based on monitored packages (packages used by users in images or explictly monitored packages by Thoth) and is something end-user does not interact with. Let's download it for our demo and store these observations into our graph database instance. 

In [3]:
import requests

SOLVER_DOCUMENT_URL = 'https://raw.githubusercontent.com/thoth-station/misc/master/examples/lockdown/resolved.json'

response = requests.get(SOLVER_DOCUMENT_URL)
response.raise_for_status()
solver_document = response.json()

Let's check for which packages was the given solver run made:

In [4]:
print(solver_document["metadata"]["arguments"]["pypi"]["requirements"].replace("\\n", "\n"))

daiquiri
flask


Also, do not forget that `Python` package resolution is dependent on environment (requirements can dynamically change based on `setup.py`/`setup.cfg` configuration). Let's check the environment for which resolution was done:

In [5]:
from  thoth.storages import SolverResultsStore

document_id = SolverResultsStore.get_document_id(solver_document)
solver_name = SolverResultsStore.get_solver_name_from_document_id(document_id)
graph.parse_python_solver_name(solver_name)

{'os_name': 'fedora', 'os_version': '29', 'python_version': '3.6'}

The above step is done internally in Thoth to obtain information about solver environment. Finally, let's sync solver results into graph database.

In [6]:
%%time

graph.sync_solver_result(solver_document)

CPU times: user 18 s, sys: 1.05 s, total: 19.1 s
Wall time: 53.9 s


As a next step, we will obtain `Pipfile` from Thamos repo with examples. In this case we have just `Pipfile`, the `Pipfile.lock` file will be generated at the bottom of this notebook by Thoth respecting environment and observations Thoth internally has to provide stack guidenance.

In [7]:
PIPFILE_URL = "https://raw.githubusercontent.com/thoth-station/thamos/master/examples/lockdown/Pipfile"

response = requests.get(PIPFILE_URL)
response.raise_for_status()
pipfile_str = response.text

The `Project` abstraction carries information about the project a user has. It wraps some additional logic to provide guidenance on project configuration as well as support for different package managers in future. This way we can transparently other managers such as `requirements.txt` or (better case) `pyproject.toml` as stated in [PEP-518](https://www.python.org/dev/peps/pep-0518/).

In [8]:
from thoth.python import Project

project = Project.from_strings(pipfile_str)
project.to_dict()

{'requirements': {'packages': {'daiquiri': '*', 'flask': '<=0.12.4'},
  'dev-packages': {},
  'source': [{'url': 'https://pypi.org/simple',
    'verify_ssl': True,
    'name': 'pypi'}],
  'requires': {'python_version': '3.6'}},
 'requirements_locked': None}

The next input for Thoth are information about environment - if they are omitted, there is done generic resolution as in case of `Pipenv` for example. If this configuration is additionally supplied, Thoth can provide additional guidenance on software stack for the given runtime environment. For example pick software stacks with great performance for CUDA or specific CPU a user uses. This configuration is part of `.thoth.yaml` file (configuration for Thoth and Thamos inside user's repositories). Let's download it and use it interactively from within this notebook:

In [9]:
import yaml

THOTH_YAML_URL = "https://raw.githubusercontent.com/thoth-station/thamos/master/examples/lockdown/.thoth.yaml"

response = requests.get(THOTH_YAML_URL)
response.raise_for_status()
config_content = yaml.load(response.text)

config_content

{'host': 'test.thoth-station.ninja',
 'tls_verify': False,
 'requirements_format': 'pipenv',
 'runtime_environments': [{'name': 'fedora:29',
   'recommendation_type': 'latest',
   'operating_system': {'name': 'fedora', 'version': '29'}}]}

Now, let's transfer this runtime environment configuration to Thoth's internal structures which provide addional checks and report any issues with user's configuration if any. We will stick with the first runtime environment stated in the configuration file (there is one stated anyway).

In [10]:
from thoth.common import RuntimeEnvironment

runtime_environment = RuntimeEnvironment.from_dict(config_content["runtime_environments"][0])

Unknown configuration entry in the configuration file recommendation_type with value latest


The warning reported can be omitted. The `recommendation_type` configuration option in the configuration file is used in Thamos to override default `recommendation_type` if needed (for a specific runtime environment entry).

Finally, let's compute some advises:

In [11]:
%%time
%env THOTH_ADVISER_SHOW_PACKAGES=1

import logging

from thoth.common import init_logging
from thoth.adviser.python import Adviser
from thoth.adviser.enums import RecommendationType

# Adjust to DEBUG for more verbose output.
logging.getLogger("thoth.adviser").setLevel(logging.INFO)

stack_info, advised_configuration, report = Adviser.compute_on_project(
    project,
    recommendation_type=RecommendationType.LATEST,
    count=1,
    limit=1,
    dry_run=False,
    graph=graph
)

env: THOTH_ADVISER_SHOW_PACKAGES=1


2019-03-13 09:25:36,239 [24560] INFO     root:126: Logging to a Sentry instance is turned off
2019-03-13 09:25:36,240 [24560] INFO     root:148: Logging to rsyslog endpoint is turned off
2019-03-13 09:25:36,267 [24560] INFO     thoth.adviser.python.scoring:62: Using scoring function that will generate latest stacks
2019-03-13 09:25:36,271 [24560] INFO     thoth.adviser.python.dependency_graph:440: Parsing and solving direct dependencies of the requested project
2019-03-13 09:25:36,273 [24560] INFO     thoth.solver.python.python_solver:113: Parsing dependency 'daiquiri'
2019-03-13 09:25:36,383 [24560] INFO     thoth.solver.python.python_solver:113: Parsing dependency 'flask<=0.12.4'
2019-03-13 09:25:36,514 [24560] INFO     thoth.adviser.python.dependency_graph:455: Retrieving transitive dependencies of direct dependencies
2019-03-13 09:25:38,862 [24560] INFO     thoth.adviser.python.dependency_graph:558: Sorting dependencies to preserve order of generated stacks
2019-03-13 09:25:41,115 

2019-03-13 09:25:42,048 [24560] INFO     thoth.adviser.python.dependency_graph:651: 	'0.11.14' from https://pypi.org/simple
2019-03-13 09:25:42,049 [24560] INFO     thoth.adviser.python.dependency_graph:651: 	'0.11.15' from https://pypi.org/simple
2019-03-13 09:25:42,051 [24560] INFO     thoth.adviser.python.dependency_graph:651: 	'0.12' from https://pypi.org/simple
2019-03-13 09:25:42,052 [24560] INFO     thoth.adviser.python.dependency_graph:651: 	'0.12.1' from https://pypi.org/simple
2019-03-13 09:25:42,053 [24560] INFO     thoth.adviser.python.dependency_graph:651: 	'0.12.2' from https://pypi.org/simple
2019-03-13 09:25:42,055 [24560] INFO     thoth.adviser.python.dependency_graph:651: 	'0.13' from https://pypi.org/simple
2019-03-13 09:25:42,057 [24560] INFO     thoth.adviser.python.dependency_graph:651: 	'0.14' from https://pypi.org/simple
2019-03-13 09:25:42,060 [24560] INFO     thoth.adviser.python.dependency_graph:651: 	'0.14.1' from https://pypi.org/simple
2019-03-13 09:25:42,

2019-03-13 09:25:42,185 [24560] INFO     thoth.adviser.python.dependency_graph:651: 	'4.0' from https://pypi.org/simple
2019-03-13 09:25:42,186 [24560] INFO     thoth.adviser.python.dependency_graph:651: 	'4.1' from https://pypi.org/simple
2019-03-13 09:25:42,187 [24560] INFO     thoth.adviser.python.dependency_graph:651: 	'5.0' from https://pypi.org/simple
2019-03-13 09:25:42,188 [24560] INFO     thoth.adviser.python.dependency_graph:651: 	'5.1' from https://pypi.org/simple
2019-03-13 09:25:42,189 [24560] INFO     thoth.adviser.python.dependency_graph:651: 	'6.0' from https://pypi.org/simple
2019-03-13 09:25:42,190 [24560] INFO     thoth.adviser.python.dependency_graph:651: 	'6.1' from https://pypi.org/simple
2019-03-13 09:25:42,195 [24560] INFO     thoth.adviser.python.dependency_graph:651: 	'6.2' from https://pypi.org/simple
2019-03-13 09:25:42,199 [24560] INFO     thoth.adviser.python.dependency_graph:651: 	'6.3' from https://pypi.org/simple
2019-03-13 09:25:42,200 [24560] INFO    

CPU times: user 5.11 s, sys: 207 ms, total: 5.32 s
Wall time: 7.5 s


After the above call, there are present two variables `stack_info` and `report`. The stack info hods some generic reports for the provided stack (such as warnings on wrong conifiguration, warnings on direct dependencies stated and so). In this demo, we will focus more on report. The report of adviser output is a list of stacks with some additional guidenance/information or justification why a user should use the computed stack.

Each entry in report is a tuple made of two items - justification (a list of reports and information about the computed stack) and the generated stack in a form of direct dependencies as well as generated lockfile (in this case this output is serialized into `Pipfile` and `Pipfile.lock`, but later there can be supported other output formats as stated above - like `pyproject.toml`).

In [12]:
from pprint import pprint

project = report[0][1]
project.to_dict()

{'requirements': {'packages': {'daiquiri': '*', 'flask': '<=0.12.4'},
  'dev-packages': {},
  'source': [{'url': 'https://pypi.org/simple',
    'verify_ssl': True,
    'name': 'pypi-org'}],
  'requires': {'python_version': '3.6'}},
 'requirements_locked': {'_meta': {'sources': [{'url': 'https://pypi.org/simple',
     'verify_ssl': True,
     'name': 'pypi-org'}],
   'requires': {'python_version': '3.6'},
   'hash': {'sha256': '09e4195b3fc6e06bd5ba3a9a641b6bd3e3b036256850507e18e37aaf7294f148'},
   'pipfile-spec': 6},
  'default': {'flask': {'version': '==0.12.4',
    'hashes': [],
    'index': 'pypi-org'},
   'werkzeug': {'version': '==0.14.1', 'hashes': [], 'index': 'pypi-org'},
   'jinja2': {'version': '==2.10', 'hashes': [], 'index': 'pypi-org'},
   'markupsafe': {'version': '==1.1.1', 'hashes': [], 'index': 'pypi-org'},
   'itsdangerous': {'version': '==1.1.0', 'hashes': [], 'index': 'pypi-org'},
   'click': {'version': '==7.0', 'hashes': [], 'index': 'pypi-org'}},
  'develop': {}}}

And advised Thoth's configuration changes by Thoth himself (add entry `python_version` to runtime environment):

In [13]:
advised_configuration

{'python_version': '3.6'}

Having the result wrapped in the `Project` instance conforms to good API design practices and we can provide guidenance not only on locked dependencies, but also on direct dependencies and additional configuration as provided in appropriate files (`Pipfile` in this case).

You can try generated stack by placing `Pipfile` and `Pipfile.lock` files into a directory [running a Flask application](https://github.com/thoth-station/thamos/blob/master/examples/lockdown/hello.py).

In [14]:
# Pipfile

print(project.pipfile.to_string())

[packages]
daiquiri = "*"
flask = "<=0.12.4"

[dev-packages]

[[source]]
url = "https://pypi.org/simple"
verify_ssl = true
name = "pypi-org"

[requires]
python_version = "3.6"




In [15]:
# Pipfile.lock

print(project.pipfile_lock.to_string())

{
    "_meta": {
        "hash": {
            "sha256": "09e4195b3fc6e06bd5ba3a9a641b6bd3e3b036256850507e18e37aaf7294f148"
        },
        "pipfile-spec": 6,
        "requires": {
            "python_version": "3.6"
        },
        "sources": [
            {
                "name": "pypi-org",
                "url": "https://pypi.org/simple",
                "verify_ssl": true
            }
        ]
    },
    "default": {
        "click": {
            "hashes": [],
            "index": "pypi-org",
            "version": "==7.0"
        },
        "flask": {
            "hashes": [],
            "index": "pypi-org",
            "version": "==0.12.4"
        },
        "itsdangerous": {
            "hashes": [],
            "index": "pypi-org",
            "version": "==1.1.0"
        },
        "jinja2": {
            "hashes": [],
            "index": "pypi-org",
            "version": "==2.10"
        },
        "markupsafe": {
            "hashes": [],
            "index": 

Now, let's print our justification - score of stack and stack number. Thoth generates stacks from the latest first (at least tries to do that, this is not easy to accomplish given the computation restrictions we have so a simple heuristic is used). 

In [16]:
report[0][0]

[{'Latest stack': 1}, {'score': 1.0}]

Feel free to experiment with additional configuration options, e.g.:

In [17]:
# Set recommendation type to one of the following:

[e.name for e in RecommendationType]

['STABLE', 'TESTING', 'LATEST']

In [18]:
# Adjust runtime specific information (e.g. provide CPU, CUDA information, ...)

RuntimeEnvironment.from_dict({}).to_dict(without_none=False)

{'hardware': {'cpu_family': None, 'cpu_model': None},
 'operating_system': {'name': None, 'version': None},
 'python_version': None,
 'cuda_version': None,
 'name': None}

In [19]:
# Adjust version ranges of Python packages being installed. These versions are resolved using
# pip's internal algorithm, so anything which is compatible with PEP-440 (and Pipfile compatible
# for Pipfile inputs) works out of box. Note this resolution is not done by installing
# dependencies (as in case of Pip/Pipenv), but there is implemented resolver on top of
# graph database which can resolve dependencies much faster as all the data are pre-computed.

PIPFILE_STR = """

[[source]]
url = "https://pypi.org/simple"
verify_ssl = true
name = "pypi"

[packages]
daiquiri = "*"
# Any flask version:
flask = "*"

[requires]
python_version = "3.6"
"""

Project.from_strings(PIPFILE_STR).to_dict()

# Mind dependencies resolved in solver run, unknown dependencies to Thoth, obviusly, cannot be resolved by Thoth.

{'requirements': {'packages': {'daiquiri': '*', 'flask': '*'},
  'dev-packages': {},
  'source': [{'url': 'https://pypi.org/simple',
    'verify_ssl': True,
    'name': 'pypi'}],
  'requires': {'python_version': '3.6'}},
 'requirements_locked': None}

Parameter `count` limits number of stacks provided in the output, parameter `limit` limits numbef of stacks scored in total.

All of the above can be accomplished using Thamos CLI (as the above is more in-depth description what Thoth does on lower level). From user's perspective a user just installs `Thamos`, adjusts configuration via `thamos config` (automatic discovery of available hardware is performed) and issues `thamos advise`. All of the above is transparent to the user, the report is shown in a well formatted table. [Follow instructions in Thamos repository to make this happen](https://github.com/thoth-station/thamos/tree/master/examples/lockdown).

Happy hacking! ;-)

In [20]:
from thoth.lab import packages_info

# Let's state Thoth's package versions for reproducible next runs of this Jupyter Notebook.
packages_info()

2019-03-13 09:25:44,658 [24560] INFO     root:126: Logging to a Sentry instance is turned off
2019-03-13 09:25:44,660 [24560] INFO     root:148: Logging to rsyslog endpoint is turned off


,package,version,importable
0,thoth.adviser,0.3.0,True
1,thoth.analyzer,0.1.2,True
2,thoth.common,0.7.1,True
3,thoth.lab,0.0.3,True
4,thoth.package_extract,1.0.1,True
5,thoth.python,0.4.6,True
6,thoth.solver,1.1.0,True
7,thoth.storages,0.9.6,True
8,thoth.worker,0.0.2,True
